In [ ]:
!cp -r ../input/petnets/timm-0.4.12-py3-none-any.whl /kaggle/working
!pip install ./timm-0.4.12-py3-none-any.whl
!cp -r ../input/petnets/model_swin.py /kaggle/working

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import model_swin
from tqdm import tqdm
import pandas as pd

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

In [ ]:
from torch.utils.data import Dataset
import os
from torchvision import transforms
from PIL import Image
import numpy as np
import torch
import csv
from tqdm import tqdm
import pandas as pd


class MyData(Dataset):

    def __init__(self, photo_root, labels_root=None, is_train=True, use_meta=False, transforms_input=None):
        self.photo_root = photo_root  # path of data
        self.labels_root = labels_root  # root of train.csv/test.csv
        self.is_train = is_train  # 当前是否处于训练状态
        # self.use_meta = use_meta
        self.transforms = transforms_input  # 预处理方式

        self.df = pd.read_csv(labels_root)

    def __getitem__(self, index):
        # img = Image.open()
        # img_label
        # return img,img_label
        path = os.path.join(self.photo_root, self.df['Id'][index]) + '.jpg'
        img = Image.open(path)
        if self.transforms is not None:
            img = self.transforms(img)

        if self.is_train:
            label = torch.as_tensor(self.df['Pawpularity'][index]).reshape(1)
            return img, label
        else:
            return img

    def __len__(self):
        # length of dataset
        return len(self.df)

In [ ]:
import numpy as np
import torch.nn as nn
def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using device:{}".format(device))

    data_path = '../input/petfinder-pawpularity-score/test'
    test_root = '../input/petfinder-pawpularity-score/test.csv'
    state_path = ['../input/pet-swin-t/net_of_fold_1.pth',
                  '../input/pet-swin-t/net_of_fold_2.pth',
                  '../input/pet-swin-t/net_of_fold_3.pth',
                  '../input/pet-swin-t/net_of_fold_4.pth',
                  '../input/pet-swin-t/net_of_fold_5.pth']
    # state_path = './net2_19.pth'
    df_test = pd.read_csv(test_root)

    test_transform = transforms.Compose(
            [transforms.Resize([224,224]),
    #          transforms.CenterCrop(224),
             transforms.ToTensor(),
             transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)]
        )
    test_data = MyData(photo_root=data_path, labels_root=test_root, is_train=False,
                        transforms_input=test_transform)
    dataloader_test = DataLoader(test_data, batch_size=1, shuffle=True, num_workers=0)
    # net = SwinTransformer(embed_dim=192, depths=[2, 2, 18, 2], num_heads=[6, 12, 24, 48], num_classes=1000)
    nets = nn.ModuleList()
    for path in state_path:
#         net = model_swin.SwinTransformer(embed_dim=96, depths=[2, 2, 18, 2], num_heads=[3, 6, 12, 24], drop_rate=0.2, attn_drop_rate=0.2, drop_path_rate=0.2)
        net = model_swin.SwinTransformer()
        in_features = net.head.in_features
#         net.head = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_features=in_features, out_features=1))
        net.head = nn.Sequential(nn.Linear(in_features=in_features, out_features=128),
                             nn.ReLU(inplace=True),
                             nn.Dropout(0.1), 
                             nn.Linear(in_features=128, out_features=64),
                             nn.ReLU(inplace=True),
                             nn.Dropout(0.1),
                             nn.Linear(in_features=64, out_features=1))
        net.load_state_dict(torch.load(path, map_location=device))
        nets.append(net)
    nets.to(device)
    sigmoid = nn.Sigmoid()
    pred_list = []
    nets.eval()
    with torch.no_grad():
        for img in dataloader_test:
            pred_base = []
            img = img.to(device)
            for net in nets:
                pred = sigmoid(net(img))
                pred_base.append(pred.item() * 100)
            pred_list.append(np.mean(pred_base))

    df_submission = pd.DataFrame({'Id': df_test['Id'], 'Pawpularity': pred_list})
    df_submission.to_csv('submission.csv', index=False)

    print(df_submission)


In [ ]:
import numpy as np
import torch.nn as nn
def main2():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using device:{}".format(device))

    data_path = '../input/petfinder-pawpularity-score/test'
    test_root = '../input/petfinder-pawpularity-score/test.csv'
    state_path = ['../input/swin-t-nodrop/net_of_fold_1.pth',
                  '../input/swin-t-nodrop/net_of_fold_2.pth',
                  '../input/swin-t-nodrop/net_of_fold_3.pth',
                  '../input/swin-t-nodrop/net_of_fold_4.pth',
                  '../input/swin-t-nodrop/net_of_fold_5.pth',
                  '../input/swin-t-nodrop/net_of_fold_6.pth',
                  '../input/swin-t-nodrop/net_of_fold_7.pth',
                  '../input/swin-t-nodrop/net_of_fold_8.pth',
                  '../input/swin-t-nodrop/net_of_fold_9.pth',
                  '../input/swin-t-nodrop/net_of_fold_10.pth'
                 ]
    # state_path = './net2_19.pth'
    df_test = pd.read_csv(test_root)

    test_transform = transforms.Compose(
            [transforms.Resize([224,224]),
    #          transforms.CenterCrop(224),
             transforms.ToTensor(),
             transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)]
        )
    test_data = MyData(photo_root=data_path, labels_root=test_root, is_train=False,
                        transforms_input=test_transform)
    dataloader_test = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=0)
    # net = SwinTransformer(embed_dim=192, depths=[2, 2, 18, 2], num_heads=[6, 12, 24, 48], num_classes=1000)
    nets = nn.ModuleList()
    for path in state_path:
#         net = model_swin.SwinTransformer(embed_dim=96, depths=[2, 2, 18, 2], num_heads=[3, 6, 12, 24], drop_rate=0.2, attn_drop_rate=0.2, drop_path_rate=0.2)
        net = model_swin.SwinTransformer(embed_dim=96, depths=[2, 2, 18, 2], num_heads=[3, 6, 12, 24])
        in_features = net.head.in_features
#         net.head = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_features=in_features, out_features=1))
#         net.head = nn.Sequential(nn.Linear(in_features=in_features, out_features=128),
#                              nn.ReLU(inplace=True),
#                              nn.Dropout(0.1), 
#                              nn.Linear(in_features=128, out_features=64),
#                              nn.ReLU(inplace=True),
#                              nn.Dropout(0.1),
#                              nn.Linear(in_features=64, out_features=1))
        net.head = nn.Linear(in_features=in_features, out_features=1)
        net.load_state_dict(torch.load(path, map_location=device))
        nets.append(net)
    nets.to(device)
    sigmoid = nn.Sigmoid()
    pred_list = []
    nets.eval()
    with torch.no_grad():
        for img in dataloader_test:
            pred_base = None
            img = img.to(device)
            for net in nets:
                pred = sigmoid(net(img))
                if pred_base is not None:
                    pred_base += pred*100
                else:
                    pred_base = pred*100
            pred_list += (pred_base.reshape(-1)/len(nets)).tolist()

    df_submission = pd.DataFrame({'Id': df_test['Id'], 'Pawpularity': pred_list})
    df_submission.to_csv('submission.csv', index=False)

    print(df_submission)


In [ ]:
main2()